In [1]:
import pandas as pd
import numpy as np
import fuzzymatcher
import datetime
from fuzzywuzzy import fuzz

In [2]:
tm_df = pd.read_csv('scraping/tm_events.csv')
events_df = pd.read_csv('scraping/sk_events.csv')
venue_df = pd.read_csv('scraping/sk_venues.csv')

In [7]:
tm_df.shape

(3154, 26)

In [4]:
tm_df['url'] = tm_df.url.apply(lambda x: x if x[12:24] == 'ticketmaster' else np.nan)

In [6]:
tm_df.dropna(subset=['url'],inplace=True)

In [9]:
tm_df.sold_out.value_counts()

0.0    2831
1.0     323
Name: sold_out, dtype: int64

In [10]:
events_df.drop(columns=['Unnamed: 0'],inplace=True)
venue_df.drop(columns=['Unnamed: 0', 'capacity'],inplace=True)

In [11]:
events_df.head()

,id,popularity_sk,type,venue_id,headliner,support,event_time
0,38048224,0.000066,Concert,2868,Jose Negroni,Negroni's Trio,2019-08-23 20:00:00-04:00
1,38049129,0.000066,Concert,2868,Jose Negroni,Negroni's Trio,2019-09-29 17:00:00-04:00
2,37362664,0.000643,Concert,2868,Benise,NaN,2019-10-12 20:00:00-04:00
3,38998599,0.000194,Concert,2868,Enrique Chia,NaN,2019-10-13 15:00:00-04:00
4,38971387,0.000006,Concert,2868,Meme Solis,NaN,2019-10-19 20:00:00-04:00


In [12]:
venue_df.head()

,venue_name,id,street,zip,latitude,longitude,metro_area,country,city_name
0,Oracle Arena,615,7000 Coliseum Way,94621,37.751554,-122.201572,SF Bay Area,US,Oakland
1,SNHU Arena,30498,555 Elm Street,03101,42.986021,-71.463052,Manchester,US,Manchester
2,PNC Arena,1770713,1400 Edwards Mill Road,27607,35.803475,-78.721933,Raleigh,US,Raleigh
3,NRG Arena,2590108,1 Reliant Park,77054,29.684833,-95.408188,Houston,US,Houston
4,BB&T Arena,513066,500 Louie B. Nunn Dr.,41099,39.031780,-84.463310,Cincinnati,US,Highland Heights


In [14]:
songkick_df = pd.merge(events_df, venue_df,left_on='venue_id',right_on='id',how='left')

In [15]:
songkick_df.head()

,id_x,popularity_sk,type,venue_id,headliner,support,event_time,venue_name,id_y,street,zip,latitude,longitude,metro_area,country,city_name
0,38048224,0.000066,Concert,2868,Jose Negroni,Negroni's Trio,2019-08-23 20:00:00-04:00,Miami Dade County Auditorium,2868,2901 W. Flagler Street,33135,25.77248,-80.24139,Miami,US,Miami
1,38049129,0.000066,Concert,2868,Jose Negroni,Negroni's Trio,2019-09-29 17:00:00-04:00,Miami Dade County Auditorium,2868,2901 W. Flagler Street,33135,25.77248,-80.24139,Miami,US,Miami
2,37362664,0.000643,Concert,2868,Benise,NaN,2019-10-12 20:00:00-04:00,Miami Dade County Auditorium,2868,2901 W. Flagler Street,33135,25.77248,-80.24139,Miami,US,Miami
3,38998599,0.000194,Concert,2868,Enrique Chia,NaN,2019-10-13 15:00:00-04:00,Miami Dade County Auditorium,2868,2901 W. Flagler Street,33135,25.77248,-80.24139,Miami,US,Miami
4,38971387,0.000006,Concert,2868,Meme Solis,NaN,2019-10-19 20:00:00-04:00,Miami Dade County Auditorium,2868,2901 W. Flagler Street,33135,25.77248,-80.24139,Miami,US,Miami


In [16]:
songkick_df.event_time.dropna(inplace=True)
tm_df.event_date.dropna(inplace=True)

In [17]:
tm_df.event_date = pd.to_datetime(tm_df.event_date, utc=False)
songkick_df.event_time = pd.to_datetime(songkick_df.event_time,utc=False)

In [19]:
tm_df.columns

Index(['Unnamed: 0', 'Unnamed: 0_x', 'address', 'city', 'country',
       'event_date', 'event_name', 'genre', 'is_presale', 'latitude',
       'longitude', 'max_tickets', 'num_markets', 'onsale_date', 'postalCode',
       'price_max', 'price_min', 'sold_out', 'state', 'subgenre', 'url',
       'venue_name', 'Unnamed: 0_y', 'Venue', 'City', 'Capacity'],
      dtype='object')

In [20]:
songkick_df.columns

Index(['id_x', 'popularity_sk', 'type', 'venue_id', 'headliner', 'support',
       'event_time', 'venue_name', 'id_y', 'street', 'zip', 'latitude',
       'longitude', 'metro_area', 'country', 'city_name'],
      dtype='object')

In [28]:
df = fuzzymatcher.fuzzy_left_join(tm_df, songkick_df,left_on=['event_date','address','postalCode'], right_on=['event_time','street','zip'])

In [29]:
fuzzy_df = df.sort_values(by=['best_match_score'],ascending=False)

In [35]:
df = fuzzy_df[fuzzy_df.best_match_score >= -0.148236]

In [39]:
df.tail(50)

,best_match_score,__id_left,__id_right,Unnamed: 0,Unnamed: 0_x,address,city,country_left,event_date,event_name,...,event_time,venue_name_right,id_y,street,zip,latitude_right,longitude_right,metro_area,country_right,city_name
394968,-0.081576,2966_left,31619_right,25658,78,911 Pine St,Seattle,US,2019-09-13 03:00:00+00:00,Die Antwoord - House Of Zef USA Tour 2019,...,2020-03-20 20:00:00-07:00,Paramount Theatre,529,901 Pine Street,98101,47.613300,-122.331390,Seattle,US,Seattle
394716,-0.081576,2965_left,31619_right,25657,78,911 Pine St,Seattle,US,2019-09-13 03:00:00+00:00,Die Antwoord - House Of Zef USA Tour 2019,...,2020-03-20 20:00:00-07:00,Paramount Theatre,529,901 Pine Street,98101,47.613300,-122.331390,Seattle,US,Seattle
395412,-0.081576,2968_left,31619_right,25660,78,911 Pine St,Seattle,US,2019-09-13 03:00:00+00:00,Die Antwoord - House Of Zef USA Tour 2019,...,2020-03-20 20:00:00-07:00,Paramount Theatre,529,901 Pine Street,98101,47.613300,-122.331390,Seattle,US,Seattle
394499,-0.081576,2964_left,31619_right,25656,78,911 Pine St,Seattle,US,2019-09-13 03:00:00+00:00,Die Antwoord - House Of Zef USA Tour 2019,...,2020-03-20 20:00:00-07:00,Paramount Theatre,529,901 Pine Street,98101,47.613300,-122.331390,Seattle,US,Seattle
72512,-0.082053,665_left,7393_right,4727,115,1 Daily's Place,Jacksonville,US,2019-09-04 22:30:00+00:00,Peter Frampton FINALE - The Farewell Tour,...,2019-09-22 20:00:00-04:00,Orpheum Theatre,1026,1 Hamilton Place,02108,42.356250,-71.061080,Boston / Cambridge,US,Boston
300575,-0.082081,2309_left,34432_right,19728,69,777 Beach Blvd,Biloxi,US,2019-08-25 01:00:00+00:00,Purple Reign - THE Prince Tribute Show,...,2019-10-25 20:00:00-05:00,Beau Rivage Theatre,36895,875 Beach Boulevard,39530,30.393030,-88.892300,Mobile,US,Biloxi
302585,-0.082477,2320_left,34435_right,19778,119,777 Beach Blvd,Biloxi,US,2019-12-07 02:00:00+00:00,Postmodern Jukebox: A Very Postmodern Christmas,...,2019-11-02 20:00:00-05:00,Beau Rivage Theatre,36895,875 Beach Boulevard,39530,30.393030,-88.892300,Mobile,US,Biloxi
417174,-0.083720,3090_left,31619_right,26170,126,911 Pine St,Seattle,US,2019-10-14 03:00:00+00:00,Postmodern Jukebox,...,2020-03-20 20:00:00-07:00,Paramount Theatre,529,901 Pine Street,98101,47.613300,-122.331390,Seattle,US,Seattle
416684,-0.083720,3088_left,31619_right,26168,126,911 Pine St,Seattle,US,2019-10-14 03:00:00+00:00,Postmodern Jukebox,...,2020-03-20 20:00:00-07:00,Paramount Theatre,529,901 Pine Street,98101,47.613300,-122.331390,Seattle,US,Seattle
417693,-0.083720,3092_left,31619_right,26172,126,911 Pine St,Seattle,US,2019-10-14 03:00:00+00:00,Postmodern Jukebox,...,2020-03-20 20:00:00-07:00,Paramount Theatre,529,901 Pine Street,98101,47.613300,-122.331390,Seattle,US,Seattle


In [40]:
df.columns

Index(['best_match_score', '__id_left', '__id_right', 'Unnamed: 0',
       'Unnamed: 0_x', 'address', 'city', 'country_left', 'event_date',
       'event_name', 'genre', 'is_presale', 'latitude_left', 'longitude_left',
       'max_tickets', 'num_markets', 'onsale_date', 'postalCode', 'price_max',
       'price_min', 'sold_out', 'state', 'subgenre', 'url', 'venue_name_left',
       'Unnamed: 0_y', 'Venue', 'City', 'Capacity', 'id_x', 'popularity_sk',
       'type', 'venue_id', 'headliner', 'support', 'event_time',
       'venue_name_right', 'id_y', 'street', 'zip', 'latitude_right',
       'longitude_right', 'metro_area', 'country_right', 'city_name'],
      dtype='object')

In [41]:
df.columns

Index(['best_match_score', '__id_left', '__id_right', 'Unnamed: 0',
       'Unnamed: 0_x', 'address', 'city', 'country_left', 'event_date',
       'event_name', 'genre', 'is_presale', 'latitude_left', 'longitude_left',
       'max_tickets', 'num_markets', 'onsale_date', 'postalCode', 'price_max',
       'price_min', 'sold_out', 'state', 'subgenre', 'url', 'venue_name_left',
       'Unnamed: 0_y', 'Venue', 'City', 'Capacity', 'id_x', 'popularity_sk',
       'type', 'venue_id', 'headliner', 'support', 'event_time',
       'venue_name_right', 'id_y', 'street', 'zip', 'latitude_right',
       'longitude_right', 'metro_area', 'country_right', 'city_name'],
      dtype='object')

In [42]:
df.drop(columns=['__id_left','__id_right', 'Unnamed: 0','id_x','id_x'], inplace=True)

/Users/matt/.pyenv/versions/anaconda3-2019.03/lib/python3.7/site-packages/pandas/core/frame.py:4097: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [43]:
df.columns

Index(['best_match_score', 'Unnamed: 0_x', 'address', 'city', 'country_left',
       'event_date', 'event_name', 'genre', 'is_presale', 'latitude_left',
       'longitude_left', 'max_tickets', 'num_markets', 'onsale_date',
       'postalCode', 'price_max', 'price_min', 'sold_out', 'state', 'subgenre',
       'url', 'venue_name_left', 'Unnamed: 0_y', 'Venue', 'City', 'Capacity',
       'popularity_sk', 'type', 'venue_id', 'headliner', 'support',
       'event_time', 'venue_name_right', 'id_y', 'street', 'zip',
       'latitude_right', 'longitude_right', 'metro_area', 'country_right',
       'city_name'],
      dtype='object')

In [44]:
df.drop(columns=['best_match_score','type', 'venue_id',
                 'venue_name_right','latitude_right', 'longitude_right'], inplace=True)

In [45]:
df.columns

Index(['Unnamed: 0_x', 'address', 'city', 'country_left', 'event_date',
       'event_name', 'genre', 'is_presale', 'latitude_left', 'longitude_left',
       'max_tickets', 'num_markets', 'onsale_date', 'postalCode', 'price_max',
       'price_min', 'sold_out', 'state', 'subgenre', 'url', 'venue_name_left',
       'Unnamed: 0_y', 'Venue', 'City', 'Capacity', 'popularity_sk',
       'headliner', 'support', 'event_time', 'id_y', 'street', 'zip',
       'metro_area', 'country_right', 'city_name'],
      dtype='object')

In [46]:
df.to_csv('merged_df.csv')

In [47]:
df.isnull().sum()

Unnamed: 0_x          0
address               0
city                  0
country_left          0
event_date            5
event_name            0
genre                 0
is_presale            0
latitude_left         0
longitude_left        0
max_tickets        2198
num_markets           0
onsale_date           5
postalCode            0
price_max           291
price_min           291
sold_out              0
state                 0
subgenre              0
url                   0
venue_name_left       0
Unnamed: 0_y          0
Venue                 0
City                  0
Capacity              0
popularity_sk         0
headliner            17
support            1515
event_time          165
id_y                  0
street               12
zip                   0
metro_area            0
country_right         0
city_name             0
dtype: int64

In [48]:
df.drop(columns =['address', 'city', 'event_date', 'latitude_left', 'longitude_left','url','id_y'],inplace=True)

In [49]:
df.columns

Index(['Unnamed: 0_x', 'country_left', 'event_name', 'genre', 'is_presale',
       'max_tickets', 'num_markets', 'onsale_date', 'postalCode', 'price_max',
       'price_min', 'sold_out', 'state', 'subgenre', 'venue_name_left',
       'Unnamed: 0_y', 'Venue', 'City', 'Capacity', 'popularity_sk',
       'headliner', 'support', 'event_time', 'street', 'zip', 'metro_area',
       'country_right', 'city_name'],
      dtype='object')

In [50]:
df.drop(columns=[ 'street', 'zip','postalCode','event_name'],inplace=True)

In [51]:
df.isnull().sum()

Unnamed: 0_x          0
country_left          0
genre                 0
is_presale            0
max_tickets        2198
num_markets           0
onsale_date           5
price_max           291
price_min           291
sold_out              0
state                 0
subgenre              0
venue_name_left       0
Unnamed: 0_y          0
Venue                 0
City                  0
Capacity              0
popularity_sk         0
headliner            17
support            1515
event_time          165
metro_area            0
country_right         0
city_name             0
dtype: int64

In [52]:
#deal with max ticket nulls
df.max_tickets.fillna(50,inplace=True)

/Users/matt/.pyenv/versions/anaconda3-2019.03/lib/python3.7/site-packages/pandas/core/generic.py:6288: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


In [53]:
#drop without headliner
df.dropna(subset=['headliner'],inplace=True)

/Users/matt/.pyenv/versions/anaconda3-2019.03/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [54]:
#merge capacity df
capacity_df = pd.read_csv('scraping/cleaned_venues.csv')

In [57]:
# fill no supports with none
df.support.fillna('none',inplace=True)

In [58]:
df.columns

Index(['Unnamed: 0_x', 'country_left', 'genre', 'is_presale', 'max_tickets',
       'num_markets', 'onsale_date', 'price_max', 'price_min', 'sold_out',
       'state', 'subgenre', 'venue_name_left', 'Unnamed: 0_y', 'Venue', 'City',
       'Capacity', 'popularity_sk', 'headliner', 'support', 'event_time',
       'metro_area', 'country_right', 'city_name'],
      dtype='object')

In [59]:
df.drop(columns = ['Unnamed: 0_x','Unnamed: 0_x'],inplace=True)

In [60]:
#create feature of days onsale
df['date_event'] = pd.to_datetime(df.event_time,utc=True)
df['onsaleDate'] = pd.to_datetime(df.onsale_date,utc=True)
df['length_sale'] = df.date_event-df.onsaleDate

/Users/matt/.pyenv/versions/anaconda3-2019.03/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/matt/.pyenv/versions/anaconda3-2019.03/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/matt/.pyenv/versions/anaconda3-2019.03/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying t

In [61]:
df.dropna(subset=['length_sale'],inplace=True)

/Users/matt/.pyenv/versions/anaconda3-2019.03/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [62]:
df['days_to_event'] = df.length_sale.apply(lambda x: x.days)

/Users/matt/.pyenv/versions/anaconda3-2019.03/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [63]:
df.drop(columns=['event_time','onsale_date','onsaleDate', 'date_event', 'length_sale'],inplace=True)

In [69]:
df.to_csv('model_df.csv')

In [2]:
df = pd.read_csv('model_df.csv')

In [51]:
##todo match census population
census_df = pd.read_csv('census_data.csv', encoding = "ISO-8859-1")

In [52]:
df = census_df[['GC.display-label.1','respop72018']]

In [53]:
df = df.iloc[2:965]

In [54]:
df.rename(columns = {'GC.display-label.1':'area', 'respop72018': 'pop_2018'},inplace=True)

In [55]:
df['area'] = df.area.apply(lambda x: (' ').join(x.split()[:-2]))